# 01 – Labeling Sentimen dengan Gemini (LLM-Assisted Labeling)

Notebook ini digunakan untuk melakukan **labeling** sentimen komentar YouTube
terkait **dugaan korupsi proyek Kereta Cepat Whoosh (Jakarta-Bandung)**, menggunakan **Gemini API**.

## Alur Proses

1. **Load Dataset**  
   Membaca `data/raw_dataset_whoosh.csv` (hasil scraping dari 10 video YouTube, ±1000 komentar)

2. **Konfigurasi Gemini API**  
   Menggunakan API key yang disimpan di file `.env`

3. **Skema Label Sentimen**  
   - **Positif**: Komentar yang membela, mendukung, atau menilai proyek Whoosh/pemerintah secara baik terkait isu korupsi
   - **Negatif**: Komentar yang mengkritik, menuduh korupsi, atau menilai proyek/pemerintah secara buruk
   - **Netral**: Komentar yang informatif, bertanya, bercanda, atau tidak menunjukkan opini jelas

4. **Batch Processing**  
   Melabeli komentar secara bertahap (batch) dengan delay antar batch untuk menghindari rate limit API

5. **Output Final**  
   Hasil akhir disimpan ke `data/labeled_dataset_whoosh.csv`

6. **Review & Koreksi Manual** (Opsional)  
   Dataset ini dapat direview dan dikoreksi manual untuk meningkatkan akurasi

## Catatan Penting

⚠️ **Label dari Gemini = pseudo-label (label awal)**  
- Bukan kebenaran mutlak, masih perlu validasi
- Akurasi tergantung pada kualitas prompt dan kemampuan model

✅ **Kualitas akhir dikontrol dengan:**  
- Prompt engineering yang baik
- Sampling & review hasil labeling
- Koreksi manual untuk data penting (opsional)

## Parameter yang Digunakan

- **Batch Size**: 25 komentar per batch
- **Delay**: 60 detik antar batch
- **Model**: `gemini-2.0-flash`
- **Total Dataset**: ~1000 komentar dari 10 video YouTube

## File Output
```
data/
├── raw_dataset_whoosh.csv          # Input (hasil scraping)
└── labeled_dataset_whoosh.csv      # Output final (dengan kolom 'sentiment')
```

## Konteks Dataset

Dataset ini berisi komentar publik dari YouTube terkait isu dugaan korupsi pada proyek Kereta Cepat Whoosh. 
Analisis sentimen dilakukan untuk memahami opini publik terhadap isu tersebut.

## Install Dependensi

In [1]:
!pip install google-generativeai python-dotenv tqdm pandas

## Import Library

In [1]:
import os
import re
import time
import glob
import pandas as pd
from google import genai
from dotenv import load_dotenv
from tqdm import tqdm

## Konfigurasi API

In [3]:
load_dotenv(override=True)
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY is None:
    raise ValueError("GEMINI_API_KEY tidak ditemukan di file .env")

client = genai.Client(api_key=GEMINI_API_KEY)

print("=" * 60)
print("SENTIMENT LABELING - WHOOSH DATASET")
print("=" * 60)

SENTIMENT LABELING - WHOOSH DATASET


## Parameter

In [4]:
BATCH_SIZE = 25      # Jumlah komentar per batch
DELAY = 60           # Delay antar batch (detik)
RETRY_DELAY = 120    # Delay saat kena rate limit (detik)

## Path File

In [5]:
DATA_DIR = "../data"
RAW_PATH = os.path.join(DATA_DIR, "raw_dataset_whoosh.csv")
OUTPUT_PATH = os.path.join(DATA_DIR, "labeled_dataset_whoosh.csv")
CHECKPOINT_PATH = os.path.join(DATA_DIR, "checkpoint_labels.csv")

## Fungsi Klasifikasi Batch

In [6]:
def classify_sentiment_batch(comments):
    """
    Klasifikasi sentimen untuk batch komentar sekaligus.
    """
    numbered_comments = "\n".join([f"{i+1}. {c}" for i, c in enumerate(comments)])
    
    prompt = f"""
Anda adalah analis sentimen publik khusus untuk isu *dugaan korupsi proyek Kereta Cepat Whoosh* 
(Jakarta—Bandung). Tugas Anda adalah menilai apakah setiap komentar menunjukkan sentimen 
positif, netral, atau negatif *terhadap isu dugaan korupsi tersebut*.

Fokus utama:
- Nilai sentimen berdasarkan sikap komentar terhadap *dugaan korupsi Whoosh*, 
  bukan sekadar terhadap layanan Whoosh sebagai kereta cepat.

Pedoman penilaian:
1. **Positive**
   - Mendukung, membela, atau tidak percaya bahwa ada korupsi.
   - Menganggap isu korupsi tidak benar, dilebih-lebihkan, atau ada pihak yang menyebarkan hoaks.
   - Menilai proyek berjalan baik dan tidak berkaitan dengan korupsi.

2. **Negative**
   - Menyatakan bahwa proyek Whoosh memang korup, merugikan negara, penuh penyimpangan.
   - Menyalahkan pemerintah, pejabat, atau pihak tertentu terkait dugaan korupsi proyek tersebut.
   - Mengkritik biaya, pembengkakan anggaran, atau tudingan penyalahgunaan dana.

3. **Neutral**
   - Tidak menunjukkan opini jelas.
   - Hanya bertanya, bercanda, atau menceritakan informasi umum.
   - Komentar tidak relevan dengan isu korupsi.

Jumlah komentar: {len(comments)}.

Berikut daftar komentar yang harus Anda klasifikasikan sesuai urutan:

{numbered_comments}

Format jawaban:
- Jawab HANYA dengan Python list berisi label untuk setiap komentar.
- Gunakan **huruf kecil semua**: "positive", "negative", "neutral".
- Contoh format:
  ["positive", "negative", "neutral"]

Jumlah elemen dalam list HARUS tepat {len(comments)}.
Jangan tambahkan kata lain, penjelasan, alasan, nomor, atau teks di luar list.
"""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    raw_output = response.text.strip()
    
    # Bersihkan format markdown jika ada
    raw_output = re.sub(r"^```(?:python)?", "", raw_output)
    raw_output = re.sub(r"```$", "", raw_output).strip()
    
    # Parsing hasil
    try:
        labels = re.findall(r'"(.*?)"', raw_output)
        if not labels:  # Jika tidak ada tanda kutip, coba pisah berdasarkan koma
            labels = [w.strip(" []'\"\n") for w in raw_output.split(",") if w.strip()]
        
        # Validasi jumlah label
        if len(labels) != len(comments):
            print(f"    [WARNING] Jumlah label ({len(labels)}) ≠ jumlah komentar ({len(comments)})")
            if len(labels) < len(comments):
                labels += ["Netral"] * (len(comments) - len(labels))
            else:
                labels = labels[:len(comments)]
        
        return labels
        
    except Exception as e:
        print("    [ERROR] Format output tidak valid:")
        print(f"    {raw_output[:200]}...")
        return ["Netral"] * len(comments)

## Fungsi Proses Labeling

In [7]:
def process_sentiment_labeling(df, batch_size=50, delay=60):
    """
    Proses labeling sentimen dengan batch processing dan checkpoint.
    """
    all_labels = []
    
    # Cek checkpoint
    start_idx = 0
    if os.path.exists(CHECKPOINT_PATH):
        df_checkpoint = pd.read_csv(CHECKPOINT_PATH)
        start_idx = len(df_checkpoint)
        all_labels = df_checkpoint['sentiment'].tolist()
        print(f"\n[INFO] Melanjutkan dari checkpoint: {start_idx} komentar sudah dilabel")
    
    total_batches = (len(df) - start_idx + batch_size - 1) // batch_size
    
    print(f"\n[INFO] Total komentar: {len(df)}")
    print(f"[INFO] Batch size: {batch_size}")
    print(f"[INFO] Total batches: {total_batches}")
    print(f"[INFO] Starting from index: {start_idx}")
    print("=" * 60)
    
    # Loop setiap batch
    for i in range(start_idx, len(df), batch_size):
        batch_num = (i // batch_size) + 1
        
        # Ambil batch komentar
        batch_comments = df['comment'].iloc[i:i+batch_size].astype(str).tolist()
        
        print(f"\n[Batch {batch_num}/{total_batches}] Processing {len(batch_comments)} komentar...")
        
        # Retry mechanism untuk rate limit
        while True:
            try:
                labels = classify_sentiment_batch(batch_comments)
                print(f"    ✓ Berhasil! Labels: {dict(pd.Series(labels).value_counts())}")
                break  # keluar dari loop jika berhasil
                
            except Exception as e:
                error_str = str(e)
                if "RESOURCE_EXHAUSTED" in error_str or "429" in error_str:
                    print(f"    [RATE LIMIT] Terkena limit API. Menunggu {RETRY_DELAY}s...")
                    time.sleep(RETRY_DELAY)
                else:
                    print(f"    [ERROR] {e}")
                    labels = ["Netral"] * len(batch_comments)
                    break
        
        # Simpan hasil batch
        all_labels.extend(labels)
        
        # Simpan checkpoint
        df_checkpoint = pd.DataFrame({
            "video_id": df["video_id"].iloc[:len(all_labels)],
            "video_title": df["video_title"].iloc[:len(all_labels)],
            "comment_id": df["comment_id"].iloc[:len(all_labels)],
            "author": df["author"].iloc[:len(all_labels)],
            "comment": df["comment"].iloc[:len(all_labels)],
            "likes": df["likes"].iloc[:len(all_labels)],
            "published_at": df["published_at"].iloc[:len(all_labels)],
            "sentiment": all_labels
        })
        df_checkpoint.to_csv(CHECKPOINT_PATH, index=False, encoding='utf-8')
        print(f"    💾 Checkpoint saved ({len(all_labels)}/{len(df)} komentar)")
        
        # Delay antar batch
        if i + batch_size < len(df):
            print(f"    ⏳ Menunggu {delay}s sebelum batch berikutnya...")
            time.sleep(delay)
    
    # Hasil akhir
    df_result = df.copy()
    df_result['sentiment'] = all_labels
    
    return df_result

## Main Execution

In [8]:
if __name__ == "__main__":
    print("\n[1] Loading dataset...")
    df = pd.read_csv(RAW_PATH)
    print(f"    Total komentar: {len(df)}")
    
    # Filter komentar kosong jika perlu
    df = df[df['comment'].notna()].reset_index(drop=True)
    print(f"    Setelah filter: {len(df)} komentar")
    
    # Preview 5 komentar pertama
    print("\n[2] Preview data:")
    print(df[['comment']].head())
    
    # Proses labeling
    print("\n[3] Mulai proses labeling...")
    df_result = process_sentiment_labeling(
        df, 
        batch_size=BATCH_SIZE, 
        delay=DELAY
    )
    
    # Simpan hasil akhir
    print(f"\n[4] Menyimpan hasil ke {OUTPUT_PATH}...")
    df_result.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
    
    # Statistik hasil
    print("\n" + "=" * 60)
    print("HASIL LABELING FINAL")
    print("=" * 60)
    print(f"\nTotal komentar: {len(df_result)}")
    print("\nDistribusi Sentimen:")
    sentiment_counts = df_result['sentiment'].value_counts()
    print(sentiment_counts)
    print("\nPersentase:")
    print(df_result['sentiment'].value_counts(normalize=True) * 100)
    
    # Hapus checkpoint setelah selesai
    if os.path.exists(CHECKPOINT_PATH):
        os.remove(CHECKPOINT_PATH)
        print("\n✓ Checkpoint dihapus")
    
    print("\n" + "=" * 60)
    print("✅ PROSES SELESAI!")
    print("=" * 60)
    print(f"\nFile tersimpan di: {OUTPUT_PATH}")
    
    # Preview hasil
    print("\n📊 Preview hasil labeling:")
    print(df_result[['comment', 'sentiment']].head(10))


[1] Loading dataset...
    Total komentar: 1000
    Setelah filter: 1000 komentar

[2] Preview data:
                                             comment
0  Yg benci ya apa aja salah.. \nYg seneng ya mak...
1  Bandung akan miliki kereta pajajaran dgn beaya...
2  SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...
3  Jokowi, Luhut, kroni2  yg harus bertanggungjaw...
4         Yg ditangkap gorengan yg makan duduk manis

[3] Mulai proses labeling...

[INFO] Total komentar: 1000
[INFO] Batch size: 25
[INFO] Total batches: 40
[INFO] Starting from index: 0

[Batch 1/40] Processing 25 komentar...
    ✓ Berhasil! Labels: {'negative': 17, 'neutral': 7, 'positive': 1}
    💾 Checkpoint saved (25/1000 komentar)
    ⏳ Menunggu 60s sebelum batch berikutnya...

[Batch 2/40] Processing 25 komentar...
    ✓ Berhasil! Labels: {'negative': 14, 'neutral': 9, 'positive': 2}
    💾 Checkpoint saved (50/1000 komentar)
    ⏳ Menunggu 60s sebelum batch berikutnya...

[Batch 3/40] Processing 25 komentar...
    ✓ 

## Dataset Hasil Labeling dengan Gemini API

In [2]:
data_labeling = pd.read_csv("../data/labeled_dataset_whoosh.csv")

In [3]:
data_labeling

,video_id,video_title,comment_id,author,comment,likes,published_at,sentiment
0,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugy77mNqV3jn_Xf_f2x4AaABAg,@mokohardono8859,Yg benci ya apa aja salah.. \nYg seneng ya mak...,0,2025-12-03T05:33:53Z,neutral
1,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzY1qxEeVrMeskZ39V4AaABAg,@Muhidin13-kr1zl,Bandung akan miliki kereta pajajaran dgn beaya...,0,2025-12-02T09:48:07Z,neutral
2,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzlM3ejBnN9PKqB2sB4AaABAg,@MA_Alpha-l4q,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,1,2025-12-01T06:59:37Z,negative
3,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzcxKsZ3V_n222CvEF4AaABAg,@nurhasanahssi2114,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",0,2025-11-30T01:18:59Z,negative
4,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugx1UtqOkCAflakGpoh4AaABAg,@omsimon-k6k,Yg ditangkap gorengan yg makan duduk manis,0,2025-11-28T15:56:25Z,neutral
...,...,...,...,...,...,...,...,...
995,Su2ZvF0EJDI,Ada 10 Pelaku Dugaan Mark Up Whoosh? Saut: Cob...,UgwCSu6Dw4N3EJVmwd54AaABAg,@aprian2024,Sekumpulang orang2 berkebutuhan khusus 😂😂😂😂.,0,2025-11-01T06:02:20Z,neutral
996,Su2ZvF0EJDI,Ada 10 Pelaku Dugaan Mark Up Whoosh? Saut: Cob...,UgzhycaDKt2kCY5rud14AaABAg,@Mahyuzar-u4s,Lanjutkan pak said...,0,2025-11-01T05:03:15Z,neutral
997,Su2ZvF0EJDI,Ada 10 Pelaku Dugaan Mark Up Whoosh? Saut: Cob...,Ugyme8_nAdsWbYVlEzN4AaABAg,@AatSupriawijaya,Kenapa Mahfud MD memberikan Pembelaan bahwa Lu...,0,2025-11-01T04:12:53Z,negative
998,Su2ZvF0EJDI,Ada 10 Pelaku Dugaan Mark Up Whoosh? Saut: Cob...,Ugwuh8YOh2t4p9m0bBx4AaABAg,@HasanBasri-yi3ks,Dosa jokowi sudah terlalu banyak sama rakyat k...,0,2025-11-01T04:06:59Z,negative
